In [1]:
import functools
import json
import itertools
import re


import geopandas as gpd
import pandas as pd
import shapely.geometry

import ipyleaflet 


import dtv_backend.fis
import dtv_backend.network.network_utilities
import networkx as nx

# Fairway information system
The map below shows the fairway information system. You can use it to navigate over the inland waterways. If you zoom in you can find node and edge id's, that will be used later in this notebook.

In [14]:
%%html
<iframe width='100%' height='400px' src="https://api.mapbox.com/styles/v1/siggyf/cl1tbrgu8000l14pk5x8jgbyg.html?title=false&access_token=pk.eyJ1Ijoic2lnZ3lmIiwiYSI6Il8xOGdYdlEifQ.3-JZpqwUa3hydjAJFXIlMA&zoomwheel=false#9.87/51.9264/4.3768" title="dtv" style="border:none;"></iframe>

# NetworkX graph
The map above showed the visualisation of the graph. We can extract the raw network from the internet. It is stored in zenodo.

In [3]:
# We can download
url = "https://zenodo.org/record/4578289/files/network_digital_twin_v0.2.pickle?download=1"

Let's load the network. This gets it from cache or from the internet and transforms geospatial objects into shapely objects.

In [4]:
network = dtv_backend.fis.load_fis_network(url)

Let's define some sites so we can navigate by name. We'll also define function to go from location to closest node/edges. 

In [16]:
def add_graph_columns(sites_gdf: gpd.GeoDataFrame, network: nx.Graph):
    """add node (n) and edge id (e) columns to the sites geodataframe"""
    f = functools.partial(dtv_backend.network.network_utilities.find_closest_node, network)
    sites_gdf['n'] = sites_gdf['geometry'].apply(f).apply(lambda x: x[0])
    f = functools.partial(dtv_backend.network.network_utilities.find_closest_edge, network)
    sites_gdf['e'] = sites_gdf['geometry'].apply(f).apply(lambda x: x[0])
    return sites_gdf

In [7]:
def waypoint_names_to_waypoints(waypoint_names, sites_gdf):
    waypoint_nodes = sites_gdf.set_index("name").loc[waypoint_names]
    waypoints = list(waypoint_nodes['n'])
    return waypoints


In [8]:
def make_sites_gdf(sites, network):
    sites_df = pd.DataFrame(sites)
    sites_df['lat'] = sites_df['latlng'].apply(lambda x: x[0])
    sites_df['lon'] = sites_df['latlng'].apply(lambda x: x[1])
    sites_gdf = gpd.GeoDataFrame(sites_df, geometry=gpd.points_from_xy(sites_df['lon'], sites_df['lat']))
    sites_gdf = add_graph_columns(sites_gdf, network)
    return sites_gdf

In [19]:
sites = [
    {"name": "ECT", "latlng": [51.945874, 4.046348]},
    {"name": "Willemsbrug", "latlng": [51.917360, 4.495536]},
    {"name": "DP World", "latlng": [51.283516, 4.252585]},
    {"name": "Vrasenedok", "latlng": [51.255026, 4.231722]},
    {"name": "BCTN", "latlng": [51.861380, 5.830828]},
    {"name": "Duisburg", "latlng": [51.447844, 6.751859]},
    {"name": "Delfzijl", "latlng": [53.314163, 6.930146]},
    {"name": "Houtribsluizen", "latlng": [52.527602, 5.434068]},
    {"name": "Burgum", "latlng": [53.185400, 5.989221]},
    {"name": "Buiten IJ", "latlng": [52.380512, 4.963281]},
    {"name": "Prinses Beatrixsluizen", "latlng": [52.016570, 5.111557]}
]
sites_gdf = make_sites_gdf(sites, network) 

In [28]:
# We sometimes need to add extra waypoints. 
# The shortest route from Delfzijl to Rotterdam is through the North Sea.
# So we need to add some locations in between (or use more restrictions)
waypoint_names = ["ECT", "Willemsbrug", "BCTN"]
waypoint_names = ["Delfzijl",  "Buiten IJ","Prinses Beatrixsluizen", "ECT"]
waypoint_names = ["Delfzijl", "Burgum", "Houtribsluizen","Buiten IJ", "Prinses Beatrixsluizen", "Willemsbrug", "ECT"]
waypoints = waypoint_names_to_waypoints(waypoint_names, sites_gdf=sites_gdf)
waypoints

['Berth4',
 'B34914_A',
 'L22820_A',
 '8861427',
 'B28367_A',
 'B12602_B',
 '8865973']

# Compute the route
Now that we have our waypoints defined we can compute the route. This will create a geojson feature collection with general info about the route in the properties. Each feature is an edge in the node. The route is split up into segments. Each segment ends with an empty edge of length 0, so that we have nodes and edges aligned. You can filter them out with the is_stop property. 

In [29]:
route_geojson = dtv_backend.fis.get_route(waypoints, network)
route_gdf = gpd.GeoDataFrame.from_features(route_geojson)
route_geojson['properties']['total_length_m']

453304.01536113437

# Visualize results
Now that we have our map we can visualize the results

In [30]:
center = (52, 5)
m = ipyleaflet.Map(center=center, zoom=7)

layer = ipyleaflet.GeoData(geo_dataframe=route_gdf.query('not is_stop'))
m.add_layer(layer);
layer = ipyleaflet.GeoData(geo_dataframe=route_gdf.query('is_stop'))
m.add_layer(layer);

display(m)

Map(center=[52, 5], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

In [13]:
route_gdf.head()

,geometry,e,is_stop,length_m,length_m_cumsum,n,segment,source,step,structure,target
0,"LINESTRING (6.93202 53.31348, 6.93114 53.31391...","[Berth4, 8862253]",False,146.387965,146.387965,Berth4,0,Berth4,0,None,8861427
1,"LINESTRING (6.93190 53.31347, 6.93110 53.31309...","[8862253, Berth46]",False,372.058119,518.446084,8862253,0,Berth4,1,None,8861427
2,"LINESTRING (6.92873 53.31238, 6.92755 53.31208...","[Berth46, 8864359]",False,3471.722998,3990.169082,Berth46,0,Berth4,2,None,8861427
3,"LINESTRING (6.89745 53.31335, 6.89315 53.31287...","[8864359, B45431_A]",False,1478.815694,5468.984776,8864359,0,Berth4,3,None,8861427
4,"LINESTRING (6.88422 53.31140, 6.88413 53.31139)","[B45431_A, B45431_B]",False,11.058823,5480.043599,B45431_A,0,Berth4,4,"{'structure_code': 'B', 'structure_id': '45431...",8861427
